# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import warnings

# Import API key
from api_keys import geoapify_key
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vilyuchinsk,52.9306,158.4028,-8.85,88,100,2.58,RU,1707245712
1,1,guerrero negro,27.9769,-114.0611,15.96,90,100,4.28,MX,1707245727
2,2,monte santo de minas,-21.1897,-46.9803,28.05,45,100,1.49,BR,1707245906
3,3,port-aux-francais,-49.3500,70.2167,6.15,86,100,12.23,TF,1707245709
4,4,mahenge,-8.6833,36.7167,19.40,95,97,0.31,TZ,1707245906


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    color='City', 
    size='Humidity',
    scale= 0.8,
    title='City Map with Humidity',
    xlabel='Longitude', 
    ylabel='Latitude',
    hover_cols=['City', 'Humidity'],
    geo = True, tiles="OSM")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (
    (city_data_df['Max Temp'] >= 21) &  #Max temperature greater than or equal to 21 degrees Celsius
    (city_data_df['Max Temp'] <= 27) &  #Max temperature less than or equal to 27 degrees Celsius
    (city_data_df['Humidity'] <= 100) &  #Humidity less than or equal to 100%
    (city_data_df['Wind Speed'] < 4.50) & #Wind Speed less than 4.50
    (city_data_df['Cloudiness'] == 0))  #Example: Cloudiness equals 0%

#Create new DataFrame
ideal_city_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Organize the DataFrame by the 'City_ID' column in ascending order
ideal_city_df = ideal_city_df.sort_values(by='City_ID')

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,saint-pierre,-21.3393,55.4781,23.82,73,0,2.57,RE,1707245908
127,127,la'ie,21.6477,-157.9253,23.92,76,0,4.02,US,1707245755
145,145,tantoyuca,21.3500,-98.2333,22.88,45,0,2.32,MX,1707245934
184,184,tazacorte,28.6290,-17.9293,21.19,49,0,2.57,ES,1707245739
204,204,marine corps base hawaii - mcbh,21.4453,-157.7513,23.92,73,0,2.68,US,1707245946
211,211,byndoor,13.8667,74.6333,25.14,55,0,2.55,IN,1707245947
247,247,talcahuano,-36.7167,-73.1167,24.82,65,0,4.12,CL,1707245954
256,256,piti village,13.4626,144.6933,25.17,86,0,4.12,GU,1707245956
379,379,richards bay,-28.7830,32.0377,22.12,83,0,2.31,ZA,1707245982
396,396,san julian,21.0167,-102.1667,21.49,12,0,2.02,MX,1707245986


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name='')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel_Name
11,saint-pierre,RE,-21.3393,55.4781,73,
127,la'ie,US,21.6477,-157.9253,76,
145,tantoyuca,MX,21.3500,-98.2333,45,
184,tazacorte,ES,28.6290,-17.9293,49,
204,marine corps base hawaii - mcbh,US,21.4453,-157.7513,73,
211,byndoor,IN,13.8667,74.6333,55,
247,talcahuano,CL,-36.7167,-73.1167,65,
256,piti village,GU,13.4626,144.6933,86,
379,richards bay,ZA,-28.7830,32.0377,83,
396,san julian,MX,21.0167,-102.1667,12,


In [6]:
#Loop through the DataFrame Lat and Lng to view the JSON response for the next step.
##NOTE: A lot of this will be repeated in the next step, just to show the format
##in the way it is required for this assigment. I need to view the JSON object to extract the 'keys'
for index, row in hotel_df.iterrows():
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Set parameters to search for a hotel
    radius = 10000
    latitude = row['Lat']
    longitude = row['Lng']
    categories = "accommodation.hotel"
    limit = 1
    filter_geometry = f'circle:{longitude},{latitude},{radius}'
    bias = f'proximity:{longitude:.6f},{latitude:.6f}'  
    params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

    # Make an API request using the query URL
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Print the JSON response for each row
    print(f"Hotel information for {row['City']} - {row['Country']}:")
    print(json.dumps(name_address, indent=4, sort_keys=True))
    print("\n")


Hotel information for saint-pierre - RE:
{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    55.47851990000001,
                    -21.34199300027139
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Tropic Hotel",
                "address_line2": "Rue Auguste Babet, 97410 Saint-Pierre, France",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Saint-Pierre",
                "country": "France",
                "country_code": "fr",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:street": "Rue Auguste Babet",
                        "name": "Tropic Hotel",
                        "

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
# Set the radius to 10,000 meters (10 kilometers)
radius = 10000 
params = {
        'categories': categories,
        'limit': limit,
        'filter[geometry]': filter_geometry,
        'bias': bias,
        'apiKey': geoapify_key}  

categories = "accommodation.hotel"
limit = 1
filter_geometry = f'circle:{longitude},{latitude},{radius}'
bias = f'proximity:{longitude},{latitude}' 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
        hotel_name = name_address["features"][0]["properties"]["address_line1"]
        # Update the 'Hotel_Name' column in the DataFrame
        hotel_df.at[index, "Hotel_Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel_Name']]

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
la'ie - nearest hotel: State Highway 83
tantoyuca - nearest hotel: Hotel Posada San Javier
tazacorte - nearest hotel: App Leyma
marine corps base hawaii - mcbh - nearest hotel: Studio In Paradise
byndoor - nearest hotel: NH66
talcahuano - nearest hotel: Club Recreativo Armada de Chile
piti village - nearest hotel: The Cliff Hotel
richards bay - nearest hotel: SUN1 Richards Bay
san julian - nearest hotel: Carretera Estado Jalisco 355
ahmadpur - nearest hotel: Nilanga-Shirur Anantpal- Gharni Road
nova sintra - nearest hotel: Residência Ka Dencho
minuwangoda - nearest hotel: Happy Day


,City,Country,Lat,Lng,Humidity,Hotel_Name
11,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
127,la'ie,US,21.6477,-157.9253,76,State Highway 83
145,tantoyuca,MX,21.3500,-98.2333,45,Hotel Posada San Javier
184,tazacorte,ES,28.6290,-17.9293,49,App Leyma
204,marine corps base hawaii - mcbh,US,21.4453,-157.7513,73,Studio In Paradise
211,byndoor,IN,13.8667,74.6333,55,NH66
247,talcahuano,CL,-36.7167,-73.1167,65,Club Recreativo Armada de Chile
256,piti village,GU,13.4626,144.6933,86,The Cliff Hotel
379,richards bay,ZA,-28.7830,32.0377,83,SUN1 Richards Bay
396,san julian,MX,21.0167,-102.1667,12,Carretera Estado Jalisco 355


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot2 = ideal_city_df.hvplot.points('Lng', 'Lat', color='City',
                                        size='Humidity',
                                        frame_width=700,
                                        frame_height=500,
                                        title='Ideal Vacation Spot',  
                                        geo=True, 
                                        tiles="OSM", 
                                        hover_cols=['City', 'Country', 'Humidity', 'Hotel_Name'])

# Display the map
map_plot2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)